In [ ]:
import gensim
#%pip install gensim
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Use the raw URL for the file
url = "https://raw.githubusercontent.com/masta-g3/amazon_nlp/master/Cell_Phones_and_Accessories_5.json.gz"

# Read the gzip file directly
df = pd.read_json(url, lines=True)
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
...,...,...,...,...,...,...,...,...,...
194434,A1YMNTFLNDYQ1F,B00LORXVUE,eyeused2loveher,"[0, 0]",Works great just like my original one. I reall...,5,This works just perfect!,1405900800,"07 21, 2014"
194435,A15TX8B2L8B20S,B00LORXVUE,Jon Davidson,"[0, 0]",Great product. Great packaging. High quality a...,5,Great replacement cable. Apple certified,1405900800,"07 21, 2014"
194436,A3JI7QRZO1QG8X,B00LORXVUE,Joyce M. Davidson,"[0, 0]","This is a great cable, just as good as the mor...",5,Real quality,1405900800,"07 21, 2014"
194437,A1NHB2VC68YQNM,B00LORXVUE,Nurse Farrugia,"[0, 0]",I really like it becasue it works well with my...,5,I really like it becasue it works well with my...,1405814400,"07 20, 2014"


In [9]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [3]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [4]:
# remove stop words

from gensim.utils import simple_preprocess
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess(text):
    return [word for word in simple_preprocess(text) if word not in stop_words]

df['processed'] = df.reviewText.apply(preprocess)

In [5]:
model= gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [8]:
model.build_vocab(df['processed'] )

In [9]:
model.corpus_count

194439

In [10]:
model.epochs

5

In [11]:
# Train the model
model.train(df['processed'], total_examples=model.corpus_count, epochs=5)

(39459882, 43362670)

In [ ]:
#model.save('wvmodel.model')

In [12]:
model.wv.most_similar('buy')

[('purchase', 0.7563537359237671),
 ('buying', 0.7281814813613892),
 ('invest', 0.6743619441986084),
 ('order', 0.5842874050140381),
 ('purchasing', 0.5722010135650635),
 ('repurchase', 0.5700567960739136),
 ('ordering', 0.5691982507705688),
 ('reorder', 0.568302571773529),
 ('bet', 0.5646075010299683),
 ('sell', 0.5564731955528259)]

In [13]:
model.wv.most_similar('bad')

[('terrible', 0.6914511322975159),
 ('horrible', 0.6324667930603027),
 ('okay', 0.6162706613540649),
 ('good', 0.6144537925720215),
 ('guess', 0.6035608053207397),
 ('ok', 0.5931935906410217),
 ('sad', 0.5925168991088867),
 ('awful', 0.5829251408576965),
 ('sucks', 0.5643715262413025),
 ('poor', 0.5639020800590515)]

### Find out similarity score:

In [14]:
model.wv.similarity('great', 'good')

0.7742046

In [15]:
model.wv.similarity('buy', 'purchase')

0.75635374

# Average Word2Vec

In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Sample sentences (pre-tokenized)
sentences = [
    ["natural", "language", "processing"],
    ["word", "embeddings", "are", "useful"],
    ["deep", "learning", "for", "nlp"]
]

# Train or load a Word2Vec model
model = Word2Vec(sentences, min_count=1)

def average_word2vec(doc):
    """Calculate average Word2Vec vector for a document"""
    vectors = []
    for word in doc:
        if word in model.wv:
            vectors.append(model.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    return np.zeros(model.vector_size)

# Get document embeddings
doc_embeddings = [average_word2vec(doc) for doc in sentences]

In [26]:
import numpy as np
np.array(doc_embeddings).shape

(3, 100)